## FIT5196-S2-2020 assessment 1
# Task 2: Text Pre-Processing (%45)

# Index
* [A. Introduction](#A.-Introduction)
* [B. Steps for the Task (METHODOLOGY)](#B.-Steps-for-the-Task-(METHODOLOGY))
* [C. Conclusion](#C.-Conclusion)
* [D. References](#D.-References)

## A. Introduction 

Task 2,  the way I have done is done by following the below-given steps in the said order. The steps start by Importing the files, Reading the excel sheets, retrieving and converting the data into a desirable format, after which tokenization is done. It can be seen after tokenization that Unigram 100 and Bigram 100 files can be easily created just by stop word removal and stemming for unigram 100 files and by using ngram get can get bigram 100 output file.

When it comes to Vocabulary building and countVec output files, I have followed a different procedure to complete the task. I started by making the 200 bigram list and later on added it to the unigram token dictionary where by using MWETokenizer the bigrams were created if present, in that particular dictionary key-value pair. After that, I drop the records with lengths less than 3 and split my dictionary into unigram and bigram dictionary. On the newly formed unigram dictionary, context-dependent words and rare words were removed by using FreqDist, and later on, stemming is performed. 

After the above steps, the bigram dictionary and the minimized unigram dictionary are combined into one. The newly formed dictionary is the final dictionary containing only the words desirable in the vocab. Finally, the vocabulary is formed by the dictionary values and sorted to be written in the vocab output file.  Furthermore, a vector is formed by CountVectorizer to be
to be written in the countVec file.  


# B. Steps for the Task (METHODOLOGY)
* [1. Importing Libraries](#1.-Importing-Libraries)
* [2. Reading Excel Data](#2.-Reading-Excel-Data)
	* [2.1 Total Files](#2.1-Total-Files)
* [ 3. Retriving Data And Converting To Desiarable Format.](#3.-Retriving-Data-And-Converting-To-Desiarable-Format.)
	* [3.1 Drop Counts](#3.1-Drop-Counts)
* [4. Tokenisation Using RegexTokenizer](#4.-Tokenisation-Using-RegexTokenizer)
* [5.Working for 100 UNI/BIGRAM File](#5.-Working-for-100-UNI/BIGRAM-File)
    * [5.1 Output File : 100 BIGRAM](#5.1-Output-File-:-100-BIGRAM)
    * [5.2 Removing-Context-Independent-words](#5.2-Removing-Context-Independent-words)
    * [5.3 Stemming](#5.3-Stemming)
        * [5.3.1 Output File : 100 UNIGRAM](#5.3.1-Output-File-:-100-UNIGRAM)
* [6. 200 Bi-grams for Vocab](#6.-200-Bi-grams-for-Vocab)
* [7. Vocab Building](#7.-Vocab-Building)
	* [7.1 Adding the 200 bigrams to each Dictionary value](#7.1-Adding-the-200-bigrams-to-each-Dictionary-value)
	* [7.2 Running MWE for bigram tokenisation.](#7.2-Running-MWE-for-bigram-tokenisation.)
	* [7.3 Removing words of length less than 3.](#7.3-Removing-words-of-length-less-than-3.)
	* [7.4 Splitting Dictionary into bigram and unigram](#7.4-Splitting-Dictionary-into-bigram-and-unigram)
    * [7.5 Context Dependent and Rare word removal](#7.5-Context-Dependent-and-Rare-word-removal )
    * [7.6 Stemming of the unigram dictionary](#7.6-Stemming-of-the-unigram-dictionary)
    * [7.7 Making a final dictionary with unigrams and Bigrams Combined](#7.7-Making-a-final-dictionary-with-unigrams-and-Bigrams-Combined)
    * [7.8 Output File : Vocab File](#7.8-Output-File-:-Vocab-File)
    * [7.9 Output File : countVec file using CountVectorizer](#7.9-Output-File-:-countVec-file-using-CountVectorizer)

# 1. Importing Libraries

In [ ]:
#######################################################   STEP-1  ##############################################################
import nltk
import xlrd
import langid
import pandas as pd

In [ ]:
from itertools import chain
from nltk.tokenize import MWETokenizer
from nltk.tokenize import RegexpTokenizer

In [ ]:
from nltk.util import ngrams

In [ ]:
from nltk.probability import *

In [ ]:
from __future__ import division

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# 2. Reading Excel Data

In [ ]:
#######################################################   STEP-2  ##############################################################

# Reading 30154987.xlsx data

# reading data into a dataframe
excel_data2 = pd.ExcelFile("30154987.xlsx")

In [ ]:
# throws 81 as the output means 81 files or 81 days 
len(excel_data2.sheet_names) # throws 81 as the output 

# 3. Retriving Data And Converting To Desiarable Format.

**Steps in Step 3**
   - **Step 3.a** : Reading the data in the excel data one sheet at a time. Each sheet is read into a datafarme. 
   - **Step 3.b**: The dataframe is cleansed of null values and restructured to get the column name as id,text and created_at
   - **Step 3.c** : All the tweets in the dataframe is check if they are english or not and non eglish onces are dropped. 
   - **Step 3.d** : Duplicate tweet are dropped from the dataframe
   - **Step 3.e** : Finaly the dictionary is made with the cleared dataframe.

In [ ]:
#######################################################   STEP-3  ##############################################################

# This is a dictionary initialised to get dictionary of unique tweets for further process.
data2 = {}

# tweet for checking duplicates
pre_english, pre = 0,0 # count before removing duplicates
post_english, post = 0,0 # count after removing duplicates


# itarating through the excel_data2.sheet_names
# getting the excel sheet index as i 
for i in range(len(excel_data2.sheet_names)):
    
    
    # temporary dataframe formed to input data from each excel sheet and after language analysis it is dropped into data2
    # taking the header as None 
    dfn = excel_data2.parse(i,header=None)
    
    # dropping all vacant rows and columns 
    dfn = dfn.dropna(axis='columns',how='all').dropna(axis='rows',how='all')
    # reseting index after removing null rows and columns 
    dfn = dfn.reset_index(drop=True)
    
    # taking the header of the dataframe which has id,text and created_at as the keys
    header = dfn.iloc[0]
    
    # dropping the header row 
    # and assigning the header as the column names
    dfn = dfn[1:]
    dfn = dfn.rename(columns = header)
    dfn = dfn.reset_index(drop=True)
    
    # list made to store the index of null non english tweets
    non_english_tweet_index = list()
    
    pre_english = pre_english + len(dfn)# tweet counts before non english removal
    
    # running a loop through the rows of the dataframe to find non english tweets
    for index in range(len(dfn.text)):
        
        # checking the instance of the tweets if int than append in non english list
        if isinstance(dfn.text[index],int)==True:
            non_english_tweet_index.append(index)
        
        # checking the language 
        elif langid.classify(dfn.text[index])[0]!="en":
            # non english appened to the list
            non_english_tweet_index.append(index)
     
    
    #dropping non english tweets
    dfn = dfn.drop(non_english_tweet_index)
    
    
    post_english = post_english+ len(dfn) # tweet counts after non english removal
    pre = pre + len(dfn) # tweet counts before duplicate removal
    
    # sorting data by id and dropping duplicate tweets
    dfn = dfn.sort_values('id', ascending=False)
    dfn = dfn.drop_duplicates(subset='id', keep='first')  
   
    post = post + len(dfn) # tweet counts after duplicate removal 
    
    #finaly adding the dataframe to the dictionary data2
    data2[excel_data2.sheet_names[i]]= '\n'.join(dfn.text.values)



# 3.1 Drop Counts

It can be clearly seen that the drops are high as we remove non english and and duplicate tweets.

In [ ]:
print(pre_english,"tweet counts before non english removal")
print(post_english,  "tweet counts after non english removal") 
print(pre,"tweet counts before duplicate removal")
print(post,"tweet counts after duplicate removal")

- *162000 tweet counts before non english removal*
- *91991 tweet counts after non english removal*
- *91991 tweet counts before duplicate removal*
- *91721 tweet counts after duplicate removal*

# 4. Tokenisation Using RegexTokenizer

In [ ]:
#######################################################   STEP-4  ##############################################################

#  Buiilding the tokenisation function for the data
def tokenizeRawData(fileid): 
    tokenizer = RegexpTokenizer(r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?")
    token = tokenizer.tokenize(data2[fileid].lower())
    return (fileid, token)

# calling tokenizeRawData to the tokenise data2 to form a dictionary named tokenized_data2.
tokenized_data2 = dict(tokenizeRawData(fileid) for fileid in data2.keys())

# 5. Working for 100 UNI/BIGRAM File

The formation of 100 Bigram file is done by performing the following tasks in order keeping in respect the output from above steps.

## 5.1 Output File : 100 BIGRAM

Bigrams are directly made using ngrams from the tokenised dictionary.
After tokensation it is recommended to make a bigram file because if we do this after context or stop word removal it will lead to loss of records where we fail to find the most common bigrams for each file.

In [ ]:
#######################################################   STEP-5  ##############################################################

# 100 bigram file <student_number>_100bi
#tokenized_data2  tokensied dict
from nltk.util import ngrams
with open('30154987_100bi.txt', 'w', encoding='utf-8') as f:
    for fileid in data2.keys():
        bigrams = ngrams(tokenized_data2[fileid], n = 2)
        fdbigram = FreqDist(bigrams)
        f.write(fileid+':'+ str(fdbigram.most_common(100))+"\n")


## 5.2 Removing Context-Independent words

For Unigram 100 file context-independent words are removed because taking them would lead to unnecessary stop word counts which will overcome other meaningful words.

In [ ]:
# reading the stop words from the given txt file and adding it to a list
stopwords = []
with open('./stopwords_en.txt') as f:
    stopwords = f.read().splitlines()

In [ ]:
# removing Stop words from the dictionary and assiging it to a different dictionary
tokenized_data2_1 = {}
for fileid in data2.keys():
    tokenized_data2_1[fileid] = [w for w in tokenized_data2[fileid] if w not in stopwords]

# 5.3 Stemming

Stemming is performed using Porter Stemmer in this assignment for unigram creation as it's necessary for unigrams to be stemmed to reduce words with similar meaning.

In [ ]:
from nltk.stem import PorterStemmer

In [ ]:
stem = PorterStemmer()

In [ ]:
# stemming without context dependent removal
# unigram list 100 
tokenized_data2_14 = {}
for fileid in data2.keys():
    tokenized_data2_14[fileid] = [stem.stem(w) for w in tokenized_data2_1[fileid] ]

## 5.3.1 Output File : 100 UNIGRAM

The 100 unigram files are written into the output by the following way.

In [ ]:
# 100 unigrams in each file 
# <student_number>_100uni
from nltk.probability import *
# writing in the file 
with open('30154987_100uni.txt', 'w', encoding='utf-8') as f:
    for fileid in data2.keys():
        # taking the most common top 100 freq dist from each file 
        f.write(fileid+':'+ str(FreqDist(tokenized_data2_14[fileid]).most_common(100))+"\n")
      

# 6. 200 Bi-grams for Vocab

It is specified that 200 Bigrams should be included in the vocab and to be made by pmi collocation.

In [ ]:
#######################################################   STEP-6  ##############################################################

## for 200 bi-gram
all_words_tokenized_data = list(chain.from_iterable(tokenized_data2.values()))
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words(all_words_tokenized_data)
bigram_list_data = finder.nbest(bigram_measures.pmi, 200)


# 7. Vocab Building 

# 7.1 Adding the 200 bigrams to each Dictionary value

We make add the bigrams to the unigram dictionary because we want to prepare our dictionary for MWETokenizer and find if bigrams are present in the dictionary value for a particular key.

In [ ]:
#######################################################   STEP-7.1  ############################################################

# This step is performed to get the unigrams from each file for running it in the MWE tokenizer.
tokenized_data2_122 = {}
for fileid in data2.keys():
    # making a set of each key value pair  
    tokenized_data2_122[fileid] = list(set(tokenized_data2_1[fileid]))
    # now adding all the values from bigram_list_data to the dictionary 
    tokenized_data2_122[fileid] = tokenized_data2_122[fileid] + bigram_list_data
  

# 7.2 Running MWE for bigram tokenisation.

Finding the bigrams present and converting them to tokens which are joined with underscore is what MWETokenizer does. 

In [ ]:
#######################################################   STEP-7.2  ############################################################

tokenized_data2_122_MWE = {}

from nltk.tokenize import MWETokenizer
### write your code below
for fileid in data2.keys():
    mwe_tokenizer = MWETokenizer(tokenized_data2_122[fileid]) # passing dictionary value with bigrams and unigram set
    tokenized_data2_122_MWE[fileid] = mwe_tokenizer.tokenize(tokenized_data2_1[fileid]) # passing dictionary to be tokenized
    

- tokenized_data2_122_MWE is the dictionary with bigram and unigram tokens.

# 7.3 Removing words of length less than 3.

Now as the above-created dictionary has both unigrams and bigrams it's clear that we can now remove token with lengths less than 3. The reason it is done now is because stemming being done later on. Stemming is always done after removing irrelevant words.

In [ ]:
#######################################################   STEP-7.3  ############################################################

# removing words  than 3 length
tokenized_data2_more_than3 ={}

for fileid in data2.keys():
    tokenized_data2_more_than3[fileid] = [w for w in tokenized_data2_122_MWE[fileid] if len(w)>=3]
    

# 7.4 Splitting Dictionary into bigram and unigram

Splitting of the dictionary is done because context dependent and rare words ar to be removed from the unigrams. 
2 dictionaries are made with unigrams and bigrams in it.

In [ ]:
#######################################################   STEP-7.4  ############################################################

# Spliting dictionary into 2 parts dictionaries 
bigram_dict_2 ={} # dictionary containing bigrams
non_bigram_dict ={} # dictionary containing unigrams

for fileid in data2.keys():
    bigram_dict_2[fileid]= [w for w in tokenized_data2_more_than3[fileid] if "_" in w] # if _ than a bigram
    non_bigram_dict[fileid] =  [w for w in tokenized_data2_more_than3[fileid] if "_" not in w]
            

# 7.5 Context Dependent and Rare word removal 

From unigram dictionary rare words and context dependent words are removed.

In [ ]:
#######################################################   STEP-7.5  ############################################################

# Removal from Unigram Dictionary
# making a list of all words to make a documnet frequency
non_clean_word_list = list(chain.from_iterable([set(value) for value in non_bigram_dict.values()]))

# Getting the document frequency of each unigram
fd_of_all_words = FreqDist(non_clean_word_list)
df_list_words_data = fd_of_all_words.most_common()

The following code finds the rare words and context dependent words in one step.

In [ ]:

# making of list of words which are less than 5 or greater than 60
words_context_dependent = list()

# iterating through the list df_list_words_data
for i in range(len(df_list_words_data)):
    # df_list_words_data[i][1] gives the document freq of the word
    if df_list_words_data[i][1] <5 or df_list_words_data[i][1] >60:
        # append in the list
        words_context_dependent.append(df_list_words_data[i][0])


In [ ]:
# converting list into set for faster processing 
words_context_dependent_set = set(words_context_dependent)

# making of dictionary with uni gram which are more than length 3 
#and have document frequency more than 5 or less than 60
uni_non_context_independent = {}
for fileid in data2.keys():
    # removing words not in words_context_dependent_set
    uni_non_context_independent[fileid] = [w for w in non_bigram_dict[fileid] if w not in words_context_dependent_set]
    

# 7.6 Stemming of the unigram dictionary

Finally, we proceed with unigram stemming to find the relevant unigrams for the vocab.

In [ ]:
#######################################################   STEP-7.6  ############################################################

# stemming the file uni gram dictinary using porter stemmer 
# stemming should only be done for unigrams
stemmed_uni_grams_final = {}

for fileid in data2.keys():
    # storing the stemmed value in the dictinary
    stemmed_uni_grams_final[fileid] = [stem.stem(w) for w in uni_non_context_independent[fileid]]
   


# 7.7 Making a final dictionary with unigrams and Bigrams Combined

No, we can finally make our dictionary containing relevant vocabs for unigrams and bigrams.

In [ ]:
#######################################################   STEP-7.7  ############################################################

# the tokens in the dictionary are from unigrams and bigrams
# all tokens are present in the vocab
final_uni_bi_dict2 = {}

for i in data2.keys():
    # taking the values of unigram dictinary
    final_uni_bi_dict2[i]=[w for w in stemmed_uni_grams_final[i]] 
    # taking the values from bigram dictionary
    final_uni_bi_dict2[i] =final_uni_bi_dict2[i]+  bigram_dict_2[i]
        

In [ ]:
# taking the values of the final dictionary to form  a vocab
final_uni_bi_dict_list = list(chain.from_iterable([set(value) for value in final_uni_bi_dict2.values()]))

Making a sorted list to be used for printing in the vocab file.

In [ ]:
# making a sorted list out of the vocab list
final_uni_bi_dict_set = set(final_uni_bi_dict_list)
final_uni_bi_dict_set = sorted(final_uni_bi_dict_set) 
final_uni_bi_dict_set = list(final_uni_bi_dict_set)

# 7.8 Output File : Vocab File

Forming the vocab file by writing the vocab list formed earlier.

In [ ]:
#######################################################   STEP-7.8  ############################################################

# now we can write the vocab list into the vocab file
# our vocan list is final_uni_bi_dict_set
from nltk.probability import *
with open('30154987_vocab.txt', 'w', encoding='utf-8') as f:
    for index in range(len(final_uni_bi_dict_set)):
        # writing the word along with it's index in the file
        f.write(final_uni_bi_dict_set[index]+':'+ str(index)+"\n")
      

# 7.9 Output File : countVec file using CountVectorizer

For getting the countVec file we need countVectorizer to be used.

In [ ]:
#######################################################   STEP-7.9  ############################################################

# Initializing the vectorizer
vectorizer = CountVectorizer(analyzer="word")

# fitting the final corpus into the the vectorizer
data_features = vectorizer1.fit_transform([' '.join(value) for value in final_uni_bi_dict2.values()])

In [ ]:
# output using sparse matrix
with open('30154987_countVec.txt', 'w', encoding='utf-8') as f:
    dict_index = 0
    for i in final_uni_bi_dict2.keys():
        start = i
        for word, count in zip(final_uni_bi_dict_set, data_features1.toarray()[dict_index]):
            if count > 0:
                start = start +","+str(final_uni_bi_dict_set.index(word))+":"+str(count)
        dict_index = dict_index + 1
        f.write(start+"\n")

# C. Conclusion

The steps performed led to a vocabulary that was perfect to be used for further analysis. And the formation of new words that people use.
By not removing highly occurring words it's impossible to find new words that are being by humans. It's important to explore new and coming words at the earliest so that  we can expand our knowledge in the domain of language processing and get new information.

# D. References

- Tutorials form Week 1 to 6. (FIT5196)
- Lecture resources form week 1 to 6.(FIT5196)